In [ ]:
import pandas as pd, numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
subm = pd.read_csv('../input/sample_submission.csv')

### Looking at the data
The training data contains a row per comment, with an id, the text of the comment, and 6 different labels that we'll try to predict.

In [ ]:
train.head()

In [ ]:
lens = train.comment_text.str.len()
lens.mean(), lens.std(), lens.max()

In [ ]:
lens.hist();